**Abstract Base Class**

Defining an Abstract Base Class (ABC) is a way of producing a contract between the class implementers and the callers. They ensure that derived classes implement particular methods from the base class. It isn't just a list of method names, but a shared understanding of what those methods should do. If you inherit from this ABC, you are promising to follow all the rules described in the comments. There is a contract between a class and its callers. The class promises to do certain things and have certain properties. At a very low level, the contract might include the name of a method or its number of parameters. In a staticly-typed language, that contract would actually be enforced by the compiler. In Python, you can use EAFP or introspection to confirm that the unknown object meets this expected contract.

There are also higher-level, semantic promises in the contract. For example, if there is a \__str\__() method, it is expected to return a string representation of the object. Python's duck-typing has many advantages in flexibility over static-typing, but it doesn't solve all the problems. ABCs offer an intermediate solution. Their real power lies in the way they allow you to customise the behaviour of isinstance and issubclass. (\__subclasshook\__ is basically a friendlier API on top of Python's \__instancecheck\__ and \__subclasscheck\__ hooks.) Adapting built-in constructs to work on custom types is very much part of Python's philosophy. Here is how collections.Container is defined in the standard library:

In [65]:
from abc import ABCMeta, abstractmethod
import collections

class Container(metaclass=ABCMeta):
    __slots__ = ()

    @abstractmethod
    def __contains__(self, x):
        return False

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Container:
            if any("__contains__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented
    
# This definition of __subclasshook__ says that any class with a __contains__ attribute is considered to be a 
# subclass of Container, even if it doesn't subclass it directly. So I can write this:

class ContainAllTheThings(object):
    def __contains__(self, item):
        return True
    
issubclass(ContainAllTheThings, collections.Container)
isinstance(ContainAllTheThings(), collections.Container)

True

In other words, if you implement the right interface, you're a subclass! ABCs provide a formal way to define interfaces in Python, while staying true to the spirit of duck-typing.  The meaning of \__subclasshook\__ is "any class which satisfies this predicate is considered a subclass for the purposes of isinstance and issubclass checks, regardless of whether it was registered with the ABC, and regardless of whether it's a direct subclass". This method should return True, False or NotImplemented. If it returns True, the subclass is considered a subclass of this ABC. If it returns False, the subclass is not considered a subclass of this ABC, even if it would normally be one. If it returns NotImplemented, the subclass check is continued with the usual mechanism.

I sometimes find myself writing polymorphic functions that can act on a single item or a collection of items, and I find isinstance(x, collections.Iterable) to be much more readable than hasattr(x, '\__iter\__'). I find that I rarely need to write my own ABC - I prefer to rely on duck typing - and I typically discover the need for one through refactoring. If I see a polymorphic function making a lot of attribute checks, or lots of functions making the same attribute checks, that smell suggests the existence of an ABC waiting to be extracted.

Even though an abstract base class can override the behaviour of isinstance and issubclass, it still doesn't enter the MRO of the virtual subclass. This is a potential pitfall for clients: not every object for which isinstance(x, MyABC) =True has the methods defined on MyABC:

In [70]:
class MyABC(metaclass=ABCMeta):
    def abc_method(self):
        pass
    @classmethod
    def __subclasshook__(cls, C):
        return True

class MyClass(object):
    pass

c = MyClass()
if isinstance(c, MyABC): # will be true because of the __subclasshook__ definition
    c.abc_method() 

AttributeError: 'MyClass' object has no attribute 'abc_method'

Where we have a base class which defines a common interface and several concrete implementations which do different things but have the same interface, we want to ensure that instantiating the base class is impossible and that forgetting to implement interface methods in one of the subclasses raises an error as early as possible. In the following example, we get an error upon instantiating the object for which certain methods have not been defined:

In [72]:
class Base(metaclass=ABCMeta):
    @abstractmethod
    def foo(self):
        pass

    @abstractmethod
    def bar(self):
        pass

class Concrete(Base):
    def foo(self):
        pass

assert issubclass(Concrete, Base)
c = Concrete()

TypeError: Can't instantiate abstract class Concrete with abstract methods bar

* see also 'Inheriting from collections.abc for custom container types' in python-standard-library

*** args** 

Variable positional arguments are useful in certain cases for convenience and for improving code readability by making function calls more clear and reducing visual clutter. For example, having to pass an empty list when it is not needed is cumbersome:

In [23]:
def my_func(msg, *my_list):
    if not my_list:
        print(msg)
    else:
        items = ', '.join(str(a) for a in my_list)
        print('{}, {}'.format(msg, items))
        
my_func('hello', 'good', 'bye')
my_func('hello')

hello, good, bye
hello


Calling the same function without the * operator:

    my_func('hello', ['good', 'bye'])
    
    my_func('hello', [])
    
Note that by using variable arguments with the \* operator, the arguments are turned into a tuple before being passed to the function, where they are unpacked. Note also that an issue arises in that you can't add positional arguments without every caller being updated. Those callers which are not updated will be subtely broken, giving bugs which are hard to track down as they do not raise exceptions. Therefore keyword arguments should be used instead if you want to extend functions that accept \*args.

In [32]:
def new_func(n, msg, *my_list):
    if not my_list:
        print('{}: {}'.format(n, msg))
    else:
        items = ', '.join(str(a) for a in my_list)
        print('{}: {}, {}'.format(n, msg, items))
        
new_func(1, 'hello', 'good', 'bye')
new_func('hello', 'good', 'bye') # broken but no error is raised!

1: hello, good, bye
hello: good, bye


Practically all software has some bugs; it's a matter of frequency and severity rather than absolute perfection.     The sooner you find a bug, the better: amongst other things, that it avoids wasting other people's time when they're bitten, and it makes schedules less likely to slip through extended debugging. When a bug does occur, you want to spend the minimum amount of time getting from the observed symptom to the root cause. A few techniques can help shift the numbers in our favor, including good error logging, good testing, and internal self-checks (assertions).

The **assert** statement provides a systematic way to check that the internal state of a program is as the programmer expected, with the goal of catching bugs. In particular, they're good for catching false assumptions that were made while writing the code, In addition they act as in-line documentation semewhat, by making the programmer's assumptions obvious ("explicit is better than implicit"). Assertions are particularly useful in Python because of Python's powerful and flexible dynamic typing system, e.g. we might want to check that a variable is of a certain type. Checking isinstance() should not be overused: if it quacks like a duck, there's perhaps no need to enquire too deeply into whether it really is. 

Places to consider putting assertions include checking parameter types, classes, or values, checking data structure invariants, checking "can't happen" situations after calling a function, to make sure that its return is reasonable.
The overall point is that if something does go wrong, we want to make it completely obvious as soon as possible.
It's easier to catch incorrect data at the point where it goes in than to work out how it got there later when it causes trouble. Assertions are not a substitute for unit tests or system tests, but rather a complement.

In [4]:
assert 2 + 2 == 5, "Houston we've got a problem"

AssertionError: Houston we've got a problem

**Built-in Exceptions**

In Python, all exceptions must be instances of a class that derives from BaseException. In a try statement with an except clause that mentions a particular class, that clause also handles any exception classes derived from that class (but not exception classes from which it is derived). Two exception classes that are not related via subclassing are never equivalent, even if they have the same name.

The built-in exceptions listed below can be generated by the interpreter or built-in functions. Except where mentioned, they have an “associated value” indicating the detailed cause of the error. This may be a string or a tuple of several items of information (e.g., an error code and a string explaining the code). The associated value is usually passed as arguments to the exception class’s constructor.

Using **@classmethod** as alternative constructors

In general, if you want to access a property of a class as a whole, and not the property of a specific instance of that class, use the a class method. If you want to access/modify a property associated with a specific instance of the class, then you will want to use an instance method.

In [79]:
from random import randint

class Cheese(object):
    def __init__(self, num_holes=0):
        self.number_of_holes = num_holes

    @classmethod
    def random(cls):
        return cls(randint(0, 100))

    @classmethod
    def slightly_holey(cls):
        return cls(randint(0, 33))

    @classmethod
    def very_holey(cls):
        return cls(randint(66, 100))
                       
gouda = Cheese()
emmentaler = Cheese.random()
leerdammer = Cheese.slightly_holey()
leerdammer.number_of_holes

24

In [73]:
class Date:
    def __init__(self, year, month, day): 
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def dmy(cls, day, month, year): 
        cls.year = year
        cls.month = month
        cls.day = day
        return cls(cls.year, cls.month, cls.day) 

    @classmethod
    def mdy(cls, month, day, year): 
        cls.year = year
        cls.month = month
        cls.day = day
        return cls(cls.year, cls.month, cls.day)
    
us_fmt = Date.mdy(4, 20, 2016)
us_fmt.day

20

**Context Managers** 
Context managers provide \__enter\__() and \__exit\__() methods that are invoked on entry to and exit from the body of the with statement. Some objects define standard clean-up actions to be undertaken when the object is no longer needed, regardless of whether or not the operation using the object succeeded or failed. Look at the following example, which tries to open a file and print its contents to the screen.

    for line in open("myfile.txt"):

        print(line, end="")

The problem with this code is that it leaves the file open for an indeterminate amount of time after this part of the code has finished executing. This is not an issue in simple scripts, but can be a problem for larger applications. The with statement allows objects like files to be used in a way that ensures they are always cleaned up promptly and correctly.

    with open("myfile.txt") as f:

        for line in f:
    
            print(line, end="")

After the statement is executed, the file f is always closed, even if a problem was encountered while processing the lines. Objects which, like files, provide predefined clean-up actions will indicate this in their documentation.

**Copy operations**

Assignment statements in Python do not copy objects, they create bindings between a target and an object. For collections that are mutable or contain mutable items, a copy is sometimes needed so one can change one copy without changing the other.

In [65]:
d = [2, 2, 2]
e = d[:]
d[2] = 4
e

[2, 2, 2]

The difference between shallow and deep copying is only relevant for compound objects (contain other objects, like lists or class instances)

In [66]:
import copy
a = [[2, 2, 2], [2, 2, 2]]
# shallow copy (could use copy.copy). Without the '[:]', 'b' would just point to the object that 'a' references
b = a[:]
c = copy.deepcopy(a)
a[0][2] = 4
b

[[2, 2, 4], [2, 2, 2]]

**Coroutines**

Threads give Python programmers a way to run multiple functions seemingly at the same time but have a number of issues which coroutines can work around. Coroutines let you have many seemingly simultaneous functions in your Python programs. They’re implemented as an extension to generators. Coroutines work by enabling the code consuming a generator to send a value back into the generator function after each yield expression. 

The generator function receives the value passed to the send function as the result of the corresponding yield expression. The initial call to next is required to prepare the generator for receiving the first send by advancing it to the first yield expression. Together, yield and send provide generators with a standard way to vary their next yielded value in response to external input. E.g., to implement a generator coroutine that yields the minimum value it’s been sent so far. Here the bare yield prepares the coroutine with the initial minimum value sent in from the outside. Then the generator repeatedly yields the new minimum in exchange for the next value to consider.

In [7]:
def minimize():
    current = yield
    while True:
        value = yield current
        current = min(value, current)

it = minimize()
next(it)
it.send(20)
it.send(40)
it.send(30)

20

In [8]:
it.send(4)

4

**Decorators**

In [14]:
def a_new_decorator(a_func):
    def wrapTheFunction():
        print("Before a_func()")
        a_func()
        print("After a_func()")
    return wrapTheFunction

def has_name():
    pass

@a_new_decorator
def decorated():
    print('foo')

decorated()
print(has_name, decorated)

Before a_func()
foo
After a_func()
<function has_name at 0x00000209503891E0> <function a_new_decorator.<locals>.wrapTheFunction at 0x00000209503758C8>


The decorator overrode the name and docstring of the function. This is needed for debugging etc., so to solve this use functools.wraps. The following is a use-case of a decorator, similar to what might be used to protect a route in Flask where if authorization credentials are not contained within the request object a return redirect statement is executed.

In [64]:
from functools import wraps
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated

@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())

can_run = False
print(func())

Function is running
Function will not run


**Descriptors**

*See also 'Customizing Attribute Access' in Python Data Model notes, and metaprogramming notes*

Descriptors are ways of controlling setting and accessing properties.  A descriptor is an object with any of the following methods (\__get\__, \__set\__, or \__delete\__), intended to be used via dotted-lookup as if it were a typical attribute of an instance. Both propery decorators and descriptors can be used for validating class attributes; descriptors are useful when you need to validate multiple attributes.

A descriptor class can	provide \__get\__ and \__set\__ methods that let you reuse logic (e.g. for grade validation behaviour for many different attributes in a single	class. 

    class Grade(object):
        def	__get__(*args, **kwargs):								
            # …
        def	__set__(*args, **kwargs):								
            # …
    
    class Exam(object):
        math_grade = Grade()
        writing_grade = Grade()				
        science_grade = Grade() 
        
When you assign	a property: 

    exam = Exam() 
    exam.writing_grade = 40 
    
It is interpreted as:

    Exam.__dict__[‘writing_grade’].__set__(exam, 40) 
    
When you access a property is will be interpreted as:

    print(Exam.__dict__[‘writing_grade’].__get__(exam,	Exam))
    
When an Exam *instance* doesn't have an attribute named writing_grade, Python will fall back to it's class attribute instead. If this class attribute is an object that has \__get\__ and \__set\__ methods, Python will assume you want to follow the descriptor protocol.

In the following example, the Grade class keeps track of values for each Exam instance by saving the per-instance state in a dictionary. Using a normal dictionary to store Grade values would work but it would leak memory by holding a reference to every instance of Exam ever passed to \__set\__ over the programs lifetime; WeakKeyDictionary will not hold Exam instances that are no longer in use.

In [93]:
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._value = WeakKeyDictionary()
        #self._value = {}
        
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._value.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value[instance] = value

class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [94]:
class SmallInt(object):
    def __set__(self, instance, value):
        if value > 10:
            raise TypeError("Must be small int")
        instance.small_int = float(value)
        
    def __get__(self, instance, owner):
        return instance.small_int
    
class MyClass(object):
    some_small_int = SmallInt()
    
mc = MyClass()
mc.some_small_int = 4
# Next line would raise an error, so descriptors can be used for e.g. type validation:
# mc.some_small_int = 11    

**Enumerate()** is a function which gives an index number to an item in a container

In [3]:
fruit = ('apple', 'banana', 'orange')
fave = [a for a, b in enumerate(fruit) if b[-1] == 'e']
print(fave)
print(dict(enumerate(fruit)))

[0, 2]
{0: 'apple', 1: 'banana', 2: 'orange'}


**Else clauses on loops**
Think of the else as 'no break', but note they can be unintuitive and hard to read:

In [ ]:
mylist = [1,2,3]
for i in mylist:
    print(i)
    if i == 3:
        break
else:
    print('Reached StopIteration or broke out of loop')


*Generators vs List Comprehensions*: a generator expression is better for situations where you don't really need (or want) to have a full list created in memory - like when you just want to iterate over the elements one at a time. If you are only iterating over the list, you can think of a generator expression as a lazy evaluated list comprehension

**Using __getattr__ and __setattr__ for lazy attributes**

Whereas __getattribute__ is called everytime an attribute is accessed, __getattr__ is called only when the attribute is missing from the object instances dictionary. This behaviour is useful in cases such as lazily loading schemaless data.

In [61]:
class Rect:
    def __init__(self, length):
        self.length = length

    def __getattr__(self, name):
        print("getattr Invoked")
        value = 2
        setattr(self, name, value)
        return value

    def area(self):
        return self.length * self.height
        
r = Rect(10)
r.area()
r.area() # __getattr__ not invoked since height has now been set
r.__dict__

getattr Invoked


{'height': 2, 'length': 10}

\__getattr\__ is not invoked if the instance is already present in the dictionary. If you want to access an undefined attribute everytime, then \__getattribute\__ can be overwritten as follows, note that the call to `super()` is required in order to set \__dict\__ for the instance to a dict, otherwise it would be set to an int:

In [96]:
class Rect:
    def __init__(self, length):
        self.length = length

    def __getattribute__(self, name):
        print("getattribute Invoked")
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 2
            setattr(self, name, value)
            return value
        
r = Rect(10)
r.height * r.length

getattribute Invoked
getattribute Invoked


20

**Formatted String Literals**

New in version 3.6. A formatted string literal or f-string is a string literal that is prefixed with 'f' or 'F'. These strings may contain replacement fields, which are expressions delimited by curly braces {}. While other string literals always have a constant value, formatted strings are really expressions evaluated at run time. If a conversion is specified, the result of evaluating the expression is converted before formatting using the `format()` protocol. Conversion '!s' calls `str()` on the result, '!r' calls `repr()`.

In [ ]:
name = "Fred"
f"He said his name is {name!r}."
# "He said his name is 'Fred'."

f"He said his name is {repr(name)}."  # repr() is equivalent to !r

# "He said his name is 'Fred'."

 
today = datetime(year=2017, month=1, day=27)
f"{today:%B %d, %Y}"  # using date format specifier

#'January 27, 2017'

import datetime

name = 'Fred'
age = 50
anniversary = datetime.date(1991, 10, 12)
f'My name is {name}, my age next year is {age+1}, my anniversary is {anniversary:%A, %B %d, %Y}.'

# 'My name is Fred, my age next year is 51, my anniversary is Saturday, October 12, 1991.'

f'He said his name is {name!r}.'

# "He said his name is 'Fred'."

one, two = 1, 2
mystring = string = f'{one}, {two}'

**Input/output with 'big data'**

If a file that are too big to be stored in RAM, one needs to be able to read part of the file and yield data as it is processed. `open()` does not actually read the file. It creates a handle to the file (a pointer to the start of the file). The file handle has a `read()` function, which reads the entire content of the file into memory. If you called this function and the file was big enough you would run into a `MemoryError` if you do not supply a size. By using a for loop, it is possible to process and yield one line at a time: 

    def chars_per_line(file_name):
        with open(file_name, "r") as fh:
            for line in fh: 
                yield len(line)
                
    with open("chars_per_line.txt", "w") as fh:
        for num_chars in chars_per_line("my_big_file.txt"):
            fh.write(str(num_chars) + "\n")

**Iterables and Iterators**
An iterable is an object capable of returning its members one at a time. This includes all sequences types (e.g. str, list etc.), as well as non-sequence types which includes dict, file objects and objects of any class with an __iter__() method.

In [4]:
class MyList(object):
    def __init__(self, lst):
        self.lst = lst
    def __iter__(self):
        return iter(self.lst)

my_iterable = MyList([3,4,5])
[i for i in my_iterable]

[3, 4, 5]

An iterator is an object representing a stream of data.  A classic iterator will be defined using a class with \__iter\__ and \__next\__ methods: 

In [33]:
class Reverser:
    
    def __init__(self, lst):
        self.ind = 0
        self.lst = lst
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self.ind -= 1
        try:
            return self.lst[self.ind]
        except IndexError:
            raise StopIteration
            
r = Reverser([2,3,4])
next(r)

4

Repeated calls to the iterator’s \__next\__() method (or passing it to the built-in function next()) return successive items in the stream. When no more data are available a StopIteration exception is raised. Ways to create an iterative function include: generator expressions, create a generator function (defined using yield instead of return), or by defining a class with \__iter\__ and \__next\__ methods. 

An iterator can be created from a container object (e.g. a list) by passing it to the iter() function. Repeatedly calling the next() method gives a new element each time until a StopIterationor is reached.

In [25]:
x = iter([1, 2, 3])
x.__next__()

1

As iterators have an \__iter\__ method, they may be used in most places where other iterables are accepted and so iterated over in a for loop:

In [27]:
def counter(low, high):
    n = low
    while n <= high:
        yield n
        n += 1
counts = counter(2,8)
[a for a in counts if a % 2 == 0]

[2, 4, 6, 8]

Attempting this with an iterator will just return the same exhausted iterator object used in the previous iteration pass, making it appear empty:

In [29]:
4 in counts

False

**The Import System**

The import statement combines two operations; it searches for the named module, then it binds the results of that search to a name in the local scope. The search operation of the import statement is defined as a call to the \__import\__() function, with the appropriate arguments. The return value is then used to perform the name binding operation of the import statement.

Other mechanisms for invoking the import system (such as importlib.import_module()) may choose to subvert \__import\__() and use its own solution to implement import semantics.

To find the specified module, the first place checked during import search is sys.modules. This mapping serves as a cache of all modules that have been previously imported, including the intermediate paths. sys.modules is writable. Deleting a key may not destroy the associated module (as other modules may hold references to it), but it will invalidate the cache entry for the named module, causing Python to search anew for the named module upon its next import. 

*Finders and loaders*

If the named module is not found in sys.modules, then Python’s import protocol is invoked to find and load the module. This protocol consists of two conceptual objects, finders and loaders.  Objects that implement both of these interfaces are referred to as importers - they return themselves when they find that they can load the requested module.

Python includes a number of default finders and importers. Finders do not actually load modules. If they can find the named module, they return a module spec, an encapsulation of the module’s import-related information, which the import machinery then uses when loading the module.

*Import hooks*

The import machinery is designed to be extensible; the primary mechanism for this are the import hooks. There are two types of import hooks: meta hooks and import path hooks. Import path hooks are called as part of sys.path (or package.\__path\__) processing. Meta hooks are registered by adding new finder objects to sys.meta_path, and import path hooks are registered by adding new callables to sys.path_hooks.

*The meta path*
When the named module is not found in sys.modules, Python next searches sys.meta_path, which contains a list of meta path finder objects. These finders are queried in order to see if they know how to handle the named module. Meta path finders must implement a method called find_spec() which takes three arguments: a name, an import path, and (optionally) a target module.

Python’s default sys.meta_path has three meta path finders, one that knows how to import built-in modules, one that knows how to import frozen modules, and one that knows how to import modules from an import path (i.e. the path based finder).

*Loading*

If and when a module spec is found, the import machinery will use it *and the loader it contains* when loading the module. Module loaders provide the critical function of loading: module execution. The import machinery uses a variety of information about each module during import, especially before loading. Most of the information is common to all modules. The import machinery fills in import-related module attributes, based on the module spec, before the loader excutes the module. By definition, if a module has a \__path\__ attribute, it is a package. 

*The Path Based Finder*

Python has several default meta path finders. One of these is the path based finder (PathFinder), which searches an import path, which contains a list of path entries. Each path entry names a location to search for modules.

The *path based finder* itself doesn’t know how to import anything. Instead, it traverses the individual path entries, associating each of them with a *path entry finder* that knows how to handle that particular kind of path. Path entry finders must implement the find_spec() method. The default set of path entry finders implement all the semantics for finding modules on the file system, handling special file types such as Python source code (.py files), Python byte code (.pyc files) and shared libraries (e.g. .so files). 

The path based finder provides additional hooks and protocols so that you can extend and customize the types of searchable path entries. For example, if you wanted to support path entries as network URLs, you could write a hook that implements HTTP semantics to find modules on the web. This hook (a callable) would return a path entry finder supporting the protocol described below, which was then used to get a loader for the module from the web.

sys.path contains a list of strings providing search locations for modules and packages. It is initialized from the PYTHONPATH environment variable and various other installation- and implementation-specific defaults. Entries in sys.path can name directories on the file system, zip files, and potentially other “locations” that should be searched for modules, such as URLs, or database queries.

As a meta path finder, the path based finder implements the find_spec() protocol previously described, however it exposes additional hooks that can be used to customize how modules are found and loaded from the import path.
Three variables are used by the path based finder, sys.path, sys.path_hooks and sys.path_importer_cache. The\__path\__ attributes on package objects are also used. These provide additional ways that the import machinery can be customized.

sys.path contains a list of strings providing search locations for modules and packages. It is initialized from the PYTHONPATH environment variable and various other installation- and implementation-specific defaults. Entries in sys.path can name directories on the file system, zip files, and potentially other “locations” (see the site module) that should be searched for modules, such as URLs, or database queries.

The path based finder is a meta path finder, so the import machinery begins the import path search by calling the path based finder’s find_spec() method as described previously. When the path argument to find_spec() is given, it will be a list of string paths to traverse - typically a package’s \__path\__ attribute for an import within that package. If the path argument is None, this indicates a top level import and sys.path is used.

The path based finder iterates over every entry in the search path, and for each of these, looks for an appropriate path entry finder (PathEntryFinder) for the path entry. Because this can be an expensive operation the path based finder maintains a cache mapping path entries to path entry finders. If the path entry is not present in the cache, the path based finder iterates over every callable in sys.path_hooks. Each of the path entry hooks in this list is called with a single argument, the path entry to be searched. 

**Iterators and Generators**

Generators simplifies creation of iterators. A generator is a function that produces a sequence of results instead of a single value. Each time the yield statement is executed the function generates a new value.


When a generator function is called, it returns a generator object without beginning execution of the function. A generator is also an iterator so you don’t have to worry about the iterator protocol. When next method is called for the first time, the function starts executing until it reaches yield statement. The yielded value is returned by the next() call.

In [60]:
def my_range(n):
    i = 0
    while i < n:
        yield i
        i += 1
        
m = my_range(10)
# a generator object is an iterator so you don't need to call iter() on the iterable first
next(m)

0

**Kwargs** 

All	positional	arguments	to	Python	functions can	also	be	passed	by	keyword, in any order, after the postional arguments are specified. Kwargs have several advantages over positional arguments. They make the function call more clear to readers of the code. They can handle default values. They provide a way of extending a functions parameters while maintaining compatibility with existing callers.

**Lazy Evaluation**

In eager evaluation, expressions are evaluated as they appear in the code and produce a concrete value. This model of evaluation, which Python uses, allows for easy reasoning about behaviour and debugging. In lazy evaluation, expressions are evaluated when they are needed and not when first encountered.  Such expressions produce objects representing computations to be performed later. Functions (as well as iterators and generators) can be used to produced deferred computation objects since when they are defined, the body is not immediately evaluated and is only done so if and when you call the function. 

From the print statement in the following example, you can see that the body of the function is immediately evaluated. If instead we wrap the function within another function or in a lambda instead of the function being executed to produce a value, it instead just produces an object which knows how to produce the value when requested. This can improve performance through reduced memory usage or e.g. reduce overhead from database querying.

In [53]:
def func(a, b):
    print('func called with {} and {}'.format(a, b))
    return a + b

a = func(1, 2) #eager evaluation

func called with 1 and 2


In [54]:
b = lambda: func(1, 2)
b

<function __main__.<lambda>>

In [55]:
b()

func called with 1 and 2


3